# DL Survival - Ventilation Outcomes
 Updated 21/11/21

In [32]:
import pandas as pd
import numpy as np
import math
import statistics
from datetime import datetime
from datetime import timedelta

## 1. Data cleaning

- Import MIMIC III data
- Review column unique values, assign correct data types
- Impute missing values


### 1.1: Importing data

In [2]:
df = pd.read_csv(r'C:\Users\marcel\trident3\mimic_combined.csv')

In [3]:
pd.set_option('display.max_columns', None)
print(df.shape)
df.head(10)

(12332, 110)


,Unnamed: 0,hb,subject_id,tricuspid,alt,plt,bg_temp,ventrate,insurance,creatinine,ckd,crp,insulin,specimen,hospital_expire_flag,ext_time,pud,los,admission_type,inr_1,sodium,hba1c,bilirubin_indirect,diab_cc,ethnicity,t2dm,dtoutput,fibrinogen,cabg,hadm_id,mi,admission_location,fio2,ffp,first_careunit,liver_mild,admittime,malig,pco2,height,baseexcess,rheum,po2,bleed_time,reint_time,plts,neutrophils,intime,bilirubin_direct,dischtime,bilirubin_total,infection,alp,ph,free_calcium,sbp,diab_un,hematocrit,magnesium,glucose,vent_array,smoking,cvd,pulmonary,tidalvol,chloride,rr,ggt,dod,t1dm,cryo,inr,potassium,pvd,aids,icustay_seq,copd,wcc,liver_severe,albumin,meanbp,gender,mit,dementia,arrhythmia,language,bicarb,hr,ptt,marital_status,bun,weight,reintubation,ast,last_careunit,deathtime,aortic,cardiac_index,ccf,lymphocytes,outtime,dbp,prbc,lactate,spo2,paraplegia,temp,pt,aado2,met_ca
0,0,"[{'charttime': datetime.datetime(2111, 2, 17, ...",29491,0,[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],[],Private,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,[],[],[],0,2111-02-16 13:00:00,0,1.0424,ELECTIVE,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],0,white,0,[],[],0,173798,0,PHYS REFERRAL/NORMAL DELI,[],[],CSRU,0,2111-02-16 07:15:00,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",167.64,"[{'charttime': datetime.datetime(2111, 2, 16, ...",0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],NaN,[],[],2111-02-16 10:20:19,[],2111-02-21 15:49:00,[],"[{'suspected_infection_time': None, 'antibioti...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",0,[],[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],NaN,0,[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",0,0,1,0,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",M,0,0,0,ENGL,"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],DIVORCED,"[{'charttime': datetime.datetime(2111, 2, 17, ...",69.50,0,[],CSRU,NaN,1,[],0,[],2111-02-17 11:21:23,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],0
1,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",26775,0,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],Medicare,"[{'charttime': datetime.datetime(2192, 4, 20, ...",0,[],[],[],0,2192-04-20 15:00:00,0,1.2855,ELECTIVE,"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],0,white,0,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",1,197759,0,PHYS REFERRAL/NORMAL DELI,[],[],CSRU,0,2192-04-20 07:15:00,0,"[{'charttime': datetime.datetime(2192, 4, 20, ...",157.48,"[{'charttime': datetime.datetime(2192, 4, 20, ...",0,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],NaN,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",2192-04-20 09:59:48,[],2192-04-24 13:23:00,[],"[{'suspected_infection_time': None, 'antibioti...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",0,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'starttime': datetime.datetime(2192, 4, 20, ...",0,0,0,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],NaN,0,[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",0,0,1,1,"[{'charttime': date

#### 1.1.1: Column lists

In [4]:
#view and reorder columns
cols = df.columns.tolist()
new_cols = ['Unnamed: 0','hadm_id','subject_id','gender','ethnicity','marital_status','insurance','language','aortic','mit','tricuspid',
            'pulmonary','cabg','temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index','pt','ptt',
            'inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
            'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time','albumin',
            'creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate','po2','pco2','baseexcess','ph','aado2',
            'fio2','ffp','insulin','cryo','prbc','infection','ventrate','tidalvol','vent_array','reintubation','liver_severe','liver_mild',
            'rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd','paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi',
            'dementia','first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag','admittime',
            'dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime','plt','diab_un','diab_cc',
            'dtoutput','specimen','dod']


ptinfo=['Unnamed:0','hadm_id','subject_id']

demographics=['gender','ethnicity','marital_status','insurance','language']

proceduretype=['aortic','mit','tricuspid','pulmonary','cabg']

vitals=['temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index']

labs=['pt','ptt','inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time',
'albumin','creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate']

bloodgases=['po2','pco2','baseexcess','ph','aado2','fio2']

products=['ffp','insulin','cryo','prbc','infection']

ventilation=['ventrate','tidalvol','vent_array','reintubation']

comorbidities=['liver_severe','liver_mild','rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd',
'paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi','dementia']

adm_cat=['first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag']

adm_num=['admittime','dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime']

others=['plt','diab_un','diab_cc','dtoutput','specimen','dod']

timeseries=[*vitals,*labs,*bloodgases,*products,*ventilation,'plt','dtoutput']
for x in ['weight','height','reintubation']:
    timeseries.remove(x)

In [5]:
df = df[new_cols]
df.head(10)

,Unnamed: 0,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod
0,0,173798,29491,M,white,DIVORCED,Private,ENGL,1,0,0,0,0,"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",69.50,167.64,[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],[],[],[],[],[],[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],[],"[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...","[{'charttime': datetime.datetime(2111, 2, 17, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...",[],"[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...","[{'charttime': datetime.datetime(2111, 2, 16, ...",[],[],[],[],[],[],"[{'suspected_infection_time': None, 'antibioti...",[],[],"[{'starttime': datetime.datetime(2111, 2, 16, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYS REFERRAL/NORMAL DELI,ELECTIVE,0,2111-02-16 07:15:00,2111-02-21 15:49:00,2111-02-16 10:20:19,2111-02-17 11:21:23,2111-02-16 13:00:00,NaN,1.0424,1,NaN,"[{'charttime': datetime.datetime(2111, 2, 17, ...",0,0,[],[],NaN
1,1,197759,26775,M,white,WIDOWED,Medicare,ENGL,1,0,0,0,1,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",82.00,157.48,"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],[],[],[],[],[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetime.datetime(2192, 4, 20, ...",[],[],[],"[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...","[{'charttime': datetime.datetime(2192, 4, 20, ...",[],"[{'charttime': datetim

### 1.2: Cleaning data types

#### 1.2.0: NaN assignment

In [6]:
df = df.replace('NaT',np.datetime64('NaT'))
df = df.replace(['[]','NaN',np.datetime64('NaT')],np.NaN)

#### 1.2.1: Datetime columns
+ add vent_duration column

In [7]:
# set column types as datetime
time_cols = ['admittime','dischtime','intime','outtime','reint_time','ext_time','deathtime']
for col in time_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')

#dod
df['dod'] = pd.to_datetime(df['dod'], format='%Y-%m-%d')

In [8]:
# define function for getting ventilation duration (1st ventilation)


### NOTE: NEED TO EDIT FORMULA FOR VENT DURATION BASED ON JAHAN'S FORMULA ###


def get_vent_duration(row):
    time_s = (row['ext_time']-row['intime']).total_seconds()
    if math.isnan(time_s):
        time_s = (row['deathtime']-row['intime']).total_seconds()
    time_min = time_s / 60
    time_h = time_min / 60
    return time_h

In [9]:
# create ~NEW COLUMN~ for vent_duration
df['vent_duration'] = df.apply(get_vent_duration, axis=1)

In [10]:
## CHECK FOR ROWS WHERE DEATHTIME < INTIME OR ADMITTIME
xtime_cols = ['ext_time','vent_duration','admittime','dischtime','intime','outtime','reint_time','deathtime','dod']
df.loc[df['vent_duration'] < 0][xtime_cols]

,ext_time,vent_duration,admittime,dischtime,intime,outtime,reint_time,deathtime,dod
810,NaT,-3.271944,2110-01-06 15:15:00,2110-01-06 12:00:00,2110-01-06 15:16:19,2110-01-07 07:03:31,NaT,2110-01-06 12:00:00,2110-01-06
1200,2109-07-18 11:33:00,-0.233333,2109-07-16 11:15:00,2109-07-22 12:57:00,2109-07-18 11:47:00,2109-07-19 16:53:00,NaT,NaT,NaT
1465,NaT,-0.642222,2162-04-02 12:33:00,2162-04-02 12:00:00,2162-04-02 12:38:32,2162-04-03 05:42:37,NaT,2162-04-02 12:00:00,2162-04-02
3364,NaT,-10.912500,2136-03-19 22:54:00,2136-03-19 12:00:00,2136-03-19 22:54:45,2136-03-20 05:55:41,NaT,2136-03-19 12:00:00,2136-03-19
3437,2125-10-02 12:05:00,-0.731944,2125-10-02 12:47:00,2125-10-17 14:20:00,2125-10-02 12:48:55,2125-10-05 10:53:40,NaT,NaT,NaT
3609,2130-04-27 14:33:00,-1.233333,2130-04-27 07:15:00,2130-05-03 13:21:00,2130-04-27 15:47:00,2130-05-01 10:17:00,NaT,NaT,NaT
3848,2193-05-24 10:15:00,-0.283333,2193-05-24 13:30:00,2193-05-29 13:33:00,2193-05-24 10:32:00,2193-05-27 09:50:00,NaT,NaT,NaT
4212,NaT,-3.383333,2153-10-12 09:49:00,2153-10-12 06:29:00,2153-10-12 09:52:00,2153-10-13 06:29:00,NaT,2153-10-12 06:29:00,2153-10-12


In [11]:
df[xtime_cols].dtypes

ext_time         datetime64[ns]
vent_duration           float64
admittime        datetime64[ns]
dischtime        datetime64[ns]
intime           datetime64[ns]
outtime          datetime64[ns]
reint_time       datetime64[ns]
deathtime        datetime64[ns]
dod              datetime64[ns]
dtype: object

#### 1.2.2: Demographics

In [12]:
for x in demographics:
    print(x,': ',df[x].unique())

gender :  ['M' 'F']
ethnicity :  ['white' 'unknown' 'other' 'black' 'asian' 'hispanic' 'native' 'WHITE'
 'OTHER' 'UNKNOWN' 'UNABLE TO OBTAIN' 'ASIAN' 'HISPANIC/LATINO'
 'BLACK/AFRICAN AMERICAN' 'AMERICAN INDIAN/ALASKA NATIVE']
marital_status :  ['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'UNKNOWN (DEFAULT)'
 'SEPARATED']
insurance :  ['Private' 'Medicare' 'Medicaid' 'Government' 'Self Pay' 'Other']
language :  ['ENGL' nan 'CAPE' 'RUSS' 'MAND' 'PORT' 'CAMB' 'SPAN' 'PTUN' 'CANT' 'ITAL'
 'POLI' 'PERS' 'VIET' '*GUJ' 'URDU' '*BEN' 'KORE' 'ALBA' '*LEB' 'AMER'
 'GREE' 'HAIT' 'THAI' 'HIND' 'TURK' 'GERM' 'ARAB' 'ETHI' 'ENGLISH' '?']


In [13]:
#ethnicity
df.replace({'ethnicity':
                {'unknown': np.NaN,'UNKNOWN':np.NaN,'UNABLE TO OBTAIN':np.NaN,
                'OTHER':'other','WHITE':'white','BLACK/AFRICAN AMERICAN':'black','ASIAN':'asian',
                'HISPANIC/LATINO':'hispanic','AMERICAN INDIAN/ALASKA NATIVE':'native'
                }
            }, 
            inplace=True)
print(df['ethnicity'].unique())

['white' nan 'other' 'black' 'asian' 'hispanic' 'native']


In [14]:
#marital_status
df.replace({'marital_status':
                {'UNKNOWN (DEFAULT)': np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'SEPARATED']


In [15]:
#language
df.replace({'language':
                {'ENGLISH':'ENGL','?':np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

['DIVORCED' 'WIDOWED' 'MARRIED' 'SINGLE' nan 'SEPARATED']


#### 1.2.3: ✔Procedure type

In [16]:
for x in proceduretype:
    print(x,': ',df[x].unique())

aortic :  [1 0]
mit :  [0 1]
tricuspid :  [0 1]
pulmonary :  [0 1]
cabg :  [0 1]


#### 1.2.4: **Vitals / Blood Gases / Products + infection / Ventilation


In [17]:
# wait for Jahan/others
# ventrate seems to be empty

#### 1.2.5: ✔Comorbidities

In [18]:
for x in comorbidities:
    print(x,': ',df[x].unique())

liver_severe :  [0 1]
liver_mild :  [0 1]
rheum :  [0 1]
cvd :  [0 1]
aids :  [0 1]
ckd :  [0 1]
copd :  [0 1]
arrhythmia :  [0 1]
pud :  [0 1]
smoking :  [0 1]
pvd :  [0 1]
paraplegia :  [0 1]
ccf :  [0 1]
met_ca :  [0 1]
t2dm :  [0 1]
t1dm :  [0 1]
malig :  [0 1]
mi :  [0 1]
dementia :  [0 1]


#### 1.2.6: Admissions (categorical)

In [19]:
for x in adm_cat:
    print(x,': ',df[x].unique())

first_careunit :  ['CSRU' 'CCU' 'SICU' 'MICU' 'TSICU'
 'Cardiac Vascular Intensive Care Unit (CVICU)' 'Coronary Care Unit (CCU)'
 'Medical Intensive Care Unit (MICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)' 'Trauma SICU (TSICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)' 'Neuro Intermediate']
last_careunit :  ['CSRU' 'TSICU' 'CCU' 'MICU' 'SICU'
 'Cardiac Vascular Intensive Care Unit (CVICU)' 'Coronary Care Unit (CCU)'
 'Medical Intensive Care Unit (MICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)' 'Trauma SICU (TSICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)']
admission_location :  ['PHYS REFERRAL/NORMAL DELI' 'TRANSFER FROM HOSP/EXTRAM'
 'CLINIC REFERRAL/PREMATURE' 'EMERGENCY ROOM ADMIT'
 'TRANSFER FROM OTHER HEALT' 'TRANSFER FROM SKILLED NUR'
 'PHYSICIAN REFERRAL' 'TRANSFER FROM HOSPITAL' 'EMERGENCY ROOM'
 'PROCEDURE SITE' 'WALK-IN/SELF REFERRAL' 'PACU'
 'INFORMATION N

In [20]:
#first_careunit
df.replace({'first_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['first_careunit'].unique())

['CSRU' 'CCU' 'SICU' 'MICU' 'TSICU' 'CVICU' 'MICU/SICU' 'Neuro SICU'
 'Neuro Inter']


In [21]:
#last_careunit
df.replace({'last_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['last_careunit'].unique())

['CSRU' 'TSICU' 'CCU' 'MICU' 'SICU' 'CVICU' 'MICU/SICU' 'Neuro SICU']


In [22]:
#admission_location
df.replace({'admission_location':
                {'TRANSFER FROM HOSP/EXTRAM':'TRANSFER FROM HOSPITAL',
                'PHYS REFERRAL/NORMAL DELI':'PHYSICIAN REFERRAL',
                'TRANSFER FROM SKILLED NUR':'TRANSFER FROM SKILLED NURSING FACILITY',
                'INFORMATION NOT AVAILABLE':np.NaN,
                'CLINIC REFERRAL':'CLINIC REFERRAL/PREMATURE',
                'EMERGENCY ROOM ADMIT':'EMERGENCY ROOM',
                }
            }, 
            inplace=True)
print(df['admission_location'].unique())

['PHYSICIAN REFERRAL' 'TRANSFER FROM HOSPITAL' 'CLINIC REFERRAL/PREMATURE'
 'EMERGENCY ROOM' 'TRANSFER FROM OTHER HEALT'
 'TRANSFER FROM SKILLED NURSING FACILITY' 'PROCEDURE SITE'
 'WALK-IN/SELF REFERRAL' 'PACU' nan 'AMBULATORY SURGERY TRANSFER'
 'INTERNAL TRANSFER TO OR FROM PSYCH']


#### 1.2.7: Others

In [23]:
for x in others:
    print(x,': ',df[x].unique())

plt :  ["[{'charttime': datetime.datetime(2111, 2, 17, 1, 36), 'value': 239.0}]"
 "[{'charttime': datetime.datetime(2192, 4, 20, 11, 3), 'value': 260.0}\n {'charttime': datetime.datetime(2192, 4, 20, 11, 52), 'value': 248.0}\n {'charttime': datetime.datetime(2192, 4, 21, 2, 40), 'value': 217.0}]"
 "[{'charttime': datetime.datetime(2170, 1, 1, 20, 29), 'value': 61.0}\n {'charttime': datetime.datetime(2170, 1, 2, 3, 56), 'value': 63.0}\n {'charttime': datetime.datetime(2170, 1, 3, 2, 6), 'value': 59.0}\n {'charttime': datetime.datetime(2170, 1, 4, 4, 9), 'value': 61.0}\n {'charttime': datetime.datetime(2170, 1, 5, 5, 9), 'value': 83.0}\n {'charttime': datetime.datetime(2170, 1, 6, 2, 15), 'value': 114.0}\n {'charttime': datetime.datetime(2170, 1, 7, 7, 40), 'value': 158.0}]"
 ...
 "[{'charttime': datetime.datetime(2118, 1, 29, 11, 56), 'value': 150.0}\n {'charttime': datetime.datetime(2118, 1, 29, 13, 15), 'value': 133.0}\n {'charttime': datetime.datetime(2118, 1, 30, 3, 14), 'value': 17

### Parsing time series data

In [62]:
df['vent_array'][0]

"[{'starttime': datetime.datetime(2111, 2, 16, 10, 0), 'endtime': datetime.datetime(2111, 2, 16, 13, 0), 'duration_hours': 3.0}]"

In [59]:
def ventarray_parser(value):
    int_time1=np.NaN
    ext_time1=np.NaN
    int_time2=np.NaN
    ext_time2=np.NaN
    if value == np.NaN:
        return np
    else:
        a = value
        a = a.replace('\n ','')
        a = a.replace('[','')
        a = a.replace(']','')
        a = a.replace("{'starttime': datetime.datetime",'')
        a = a.replace(" 'endtime': datetime.datetime",'')
        a = a.replace(" 'duration_hours': ",'')
        split = a.split('}')
        del split[-1]
        int_time1=np.NaN
        ext_time1=np.NaN
        int_time2=np.NaN
        ext_time2=np.NaN
        if len(split) == 1:

        elif len(split) == 2:
            asdasfsa
        else:
            raise ValueError("length of vent_array is wonky")

test_x = df['vent_array'][13]
ventarray_parser(test_x)

IndentationError: expected an indented block (Temp/ipykernel_17716/3059518929.py, line 18)

In [ ]:
def infection_parser(value, timelimit):
    if value == np.NaN:
        return np.NaN
    else:
        a = value
        a = a.replace('\n ','')
        a = a.replace('[','')
        a = a.replace(']','')
        a = a.replace("{'charttime': datetime.datetime",'')
        split = a.split('}')

In [37]:

def ts_parser(value, timelimit):
    """
    Takes single string of timeseries data in MIMIC format and returns the mean, max, min values   
    Parameters
    ----------
    value : single string of timeseries data in MIMIC format
    timelimit : time (in hours) from the first data entry to include data up to

    Returns
    -------
    avg : mean of all values within specified time period
    max_: maximum of all values within specified time period
    min_: minimum of all values within specified time period
    """
    if value == np.NaN:
        return np.NaN
    else:
        a = value
        a = a.replace('\n ','')
        a = a.replace('[','')
        a = a.replace(']','')
        a = a.replace("{'charttime': datetime.datetime",'')
        split = a.split('}')
        del split[-1]
        times = []
        values = []
        for n in range(0,len(split)):
            subsplit = split[n].split(", 'value'")
            t = datetime.strptime(subsplit[0],'(%Y, %m, %d, %H, %M)')
            times.append(t)
            v = float(subsplit[1].replace(': ',''))
            values.append(v)
        starttime = times[0]
        endtime = times[0] + timedelta(hours=timelimit)
        #find the average
        incl_values = []
        for n in range(0,len(split)):
            if times[n] > starttime and times[n] < endtime: 
                incl_values.append(values[n])
        print(incl_values)
        avg = statistics.mean(incl_values)
        max_ = max(incl_values)
        min_ = min(incl_values)
        return avg, max_, min_

test_x = df[timeseries].iloc[0,0]
ts_parser(test_x,12)

[35.88888804117838, 36.000001695421005, 35.611110263400604, 35.88888804117838, 36.222220526801216, 36.44444359673394, 36.44444359673394, 36.388888888888886]


(36.111110581292046, 36.44444359673394, 35.611110263400604)

### 1.3: Handling missing data

In [63]:
# formula for checking % missing values
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0: 'Missing Values', 1: '% Missing Values'})
    return mis_val_table_ren_columns

missing_data = missing_values_table(df)

In [64]:
#set limit and get list of variables missing above limit in `missing_cols`
missing_limit = 20
missing_cols = missing_data.loc[missing_data['% Missing Values']>missing_limit].index.tolist()
print(missing_cols)
missing_data.loc[missing_data['% Missing Values']>missing_limit]

['language', 'bg_temp', 'cardiac_index', 'pt', 'fibrinogen', 'plts', 'lymphocytes', 'neutrophils', 'alp', 'ast', 'alt', 'ggt', 'bilirubin_indirect', 'bilirubin_direct', 'bilirubin_total', 'crp', 'bleed_time', 'albumin', 'lactate', 'aado2', 'fio2', 'ffp', 'insulin', 'cryo', 'prbc', 'ventrate', 'tidalvol', 'vent_array', 'ext_time', 'reint_time', 'deathtime', 'dtoutput', 'specimen', 'dod', 'vent_duration']


,Missing Values,% Missing Values
language,3551,28.795005
bg_temp,9324,75.608174
cardiac_index,8743,70.896854
pt,4339,35.184885
fibrinogen,3911,31.714239
plts,10465,84.860525
lymphocytes,6819,55.295167
neutrophils,6820,55.303276
alp,9722,78.835550
ast,9668,78.397665


In [ ]:
missing_data.loc[time_cols,:]

In [ ]:
# option 1: delete all rows in `missing_cols` (set inplace to true to execute)

df.drop(columns=missing_cols, inplace=False)

# reset index
#df.reset_index(drop=True, inplace=True)

In [ ]:
# option 2: impute data based on median


In [ ]:
# option 3: multiple imputation